In [1]:
import numpy as np
import rasterio, session_info
from osgeo import gdal, osr
from rasterio.windows import Window
#from rasterio.crs import CRS
#from rasterio.transform import from_origin
#import os, shutil, time, csv,math
#driver = "GTiff"
driver = gdal.GetDriverByName("GTiff")
session_info.show()

## Define Functions to Update Window Extents

In [2]:
def get_window_crop(img,tile_size):
    #this function crops the image with metadata to match inference extents, as per inference cropping process
    #from rasterio docs: Window(col_off, row_off, width, height)
    x = img.width
    y = img.height
    cropx = (round(int(x/tile_size),0)*tile_size)
    cropy = (round(int(y/tile_size),0)*tile_size)
    startx = x//2-(cropx//2)
    starty=y//2-(cropy//2)
    print(startx, startx+cropx, starty, starty+cropy)
    window = Window(startx,starty,cropx,cropy)
    return window


## Set Paths and Tile Size

In [36]:
#point to original image file with the geo metadata intact
#path_to_georef_raster=r'E:/MtPolley_Project/4_GIS/2022_DEM_GANs/PyWorking/output_for_Mitra/External_Test1/cdem_fill.tif'
#path_to_georef_raster = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_ortho/2019_ort_4inf.tif'
#path_to_georef_raster = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_ortho/2019_ort_4inf_small2.tif'
#path_to_georef_raster = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_mine_site/MtP_Mine_clip_small.tif'
#path_to_georef_raster = r'F:/MtPolley/tree_counting/10cm_Resamples/2019_QCNL_10cm/D15_10cm.tif'
#path_to_georef_raster = r'D:/EO4CEWP9_UAV_Datasets/2022_UofC_AB/StonyFen/StonyFen_clip_rs.tif'
path_to_georef_raster = r'F:/MtPolley/tree_counting/10cm_Resamples/2022_UofC/StonyFen_10cm.tif'
#path_to_georef_raster = r'F:/MtPolley/tree_counting/10cm_Resamples/2021_SumacSouthON_10cm/n9_10cm.tif'
#path_to_georef_raster = r'F:/MtPolley/tree_counting/10cm_Resamples/2022_USherbrooke_QCNL/Site_20_10cm.tif'
#path_to_georef_raster = r'E:/MtPolley_Project/4_GIS/for_CNN/ortho/2023_treecounting/2019_ortho/2019_ort_clip_AI_wrkshp.tif'

#point to the inference output image that you want to write metadata to
#path_to_raster_with_data=r'E:/MtPolley_Project/4_GIS/2022_DEM_GANs/PyWorking/output_for_Mitra/Results/results_set8_clipped/SR_set8_all_in_one_clipped.tif'
#path_to_raster_with_data = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/GRSimple_xtra0_+dense_MtP_clip.tif'
#path_to_raster_with_data = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/GRSimple_xtra0_+dense_MtP_Mine_clip.tif'
#path_to_raster_with_data = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/GRSimple_xtra0_+dense_D15.tif'
#path_to_raster_with_data = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/GRSimple_xtra0_+dense_paddingTest.tif'
path_to_raster_with_data = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/newV1_StonyFen_ts128_heatmap_localNorm_gc.tif'
#path_to_raster_with_data = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/heatmap_test_newV1_ts128_heatmap_localNorm_gc.tif'

#define the name of output image
#output_raster=r'E:/MtPolley_Project/4_GIS/2022_DEM_GANs/PyWorking/output_for_Mitra/Results/JL/set8_clipped_GeoRef.tif'
output_raster = r'C:/Users/jlovitt/Pyworking/for_CNN_5/RGB_aerial/inference/newV1_StonyFen_ts128_heatmap_localNorm_gc_georef.tif' 

tile_size = 128 # for tree counting CNN tile size is usually 128 unless padding was used during inference
is_FSD = False
#tile_size = 96 # for SRGAN tile size depends on what was used in training

## Load Images, Set dtype to match Inference Image

In [37]:
raster_with_ref= rasterio.open(path_to_georef_raster)

In [38]:
input_crs = raster_with_ref.crs
input_gt  = raster_with_ref.transform

print(input_crs)
print("image width: ",raster_with_ref.width,"  image height: ",raster_with_ref.height)
print("Upper Left Image Origin= X:",input_gt[2],", Y:",input_gt[5],"Pixel size: ", input_gt[0],"m,",input_gt[4],"m")

EPSG:2956
image width:  7983   image height:  5421
Upper Left Image Origin= X: 480586.64771000005 , Y: 6246389.955510001 Pixel size:  0.09999999999999855 m, -0.09999999999993128 m


In [39]:
ras = get_window_crop(raster_with_ref,tile_size)
print(ras.width,ras.height)

23 7959 22 5398
7936 5376


In [40]:
newxo = input_gt[2]+(((raster_with_ref.width-ras.width)/2)*input_gt[0])
newyo = input_gt[5]+(((raster_with_ref.height-ras.height)/2)*input_gt[4])
print(newxo,newyo)

480588.99771 6246387.705510001


In [41]:
if is_FSD == True:
    ground_resolution = tile_size/10
else:
    ground_resolution = input_gt[0]

## Rescale image to Prediction Map Resolution (e.g. 12.8m x 12.8m pixels)

In [42]:
rescale = rasterio.transform.from_origin(newxo, newyo,ground_resolution,ground_resolution)
print(rescale)

| 0.10, 0.00, 480589.00|
| 0.00,-0.10, 6246387.71|
| 0.00, 0.00, 1.00|


In [43]:
#rescale = rasterio.transform.from_origin(input_gt[2], input_gt[5],ground_resolution,ground_resolution)
#print(rescale)

## Open Raster with Data (Inference Image)

In [44]:
raster_with_data = rasterio.open(path_to_raster_with_data)

raster_array_with_data = raster_with_data.read()
dtype=raster_array_with_data.dtype

## Get Data Dimensions

In [45]:
print("image width: ",raster_with_data.width,"  image height: ",raster_with_data.height)
height = raster_with_data.height
width = raster_with_data.width

image width:  7983   image height:  5421


window = get_window_crop(raster_with_ref,tile_size) 
print(window.height,window.width)

## Write Metadata to Inference Image

In [46]:
if is_FSD == True:
    
    with raster_with_ref as src:
        kwargs = raster_with_ref.meta.copy()
        print(kwargs)
    
        kwargs.update({
            'height': raster_with_data.height,
            'width': raster_with_data.width,
            'dtype': dtype,
            'count':raster_with_data.count,
            'transform':rescale})
        print(kwargs)
    
        with rasterio.open(output_raster, 'w', **kwargs) as dst: #defining output image and referencing updated kwargs
            dst.write(raster_with_data.read())
            
else:
    with raster_with_ref as src:
        kwargs = raster_with_ref.meta.copy()
        print(kwargs)
    
        kwargs.update({
            'height': raster_with_data.height,
            'width': raster_with_data.width,
            'dtype': dtype,
            'count':raster_with_data.count,
            'transform':src.transform})
        print(kwargs)
    
        with rasterio.open(output_raster, 'w', **kwargs) as dst: #defining output image and referencing updated kwargs
            dst.write(raster_with_data.read())

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 7983, 'height': 5421, 'count': 4, 'crs': CRS.from_epsg(2956), 'transform': Affine(0.09999999999999855, 0.0, 480586.64771000005,
       0.0, -0.09999999999993128, 6246389.955510001)}
{'driver': 'GTiff', 'dtype': dtype('float32'), 'nodata': None, 'width': 7983, 'height': 5421, 'count': 1, 'crs': CRS.from_epsg(2956), 'transform': Affine(0.09999999999999855, 0.0, 480586.64771000005,
       0.0, -0.09999999999993128, 6246389.955510001)}


with raster_with_ref as src: #write updated geospatial metadata to inference image
    #window = get_window_crop(raster_with_ref,tile_size) #update window extents by applying crop

    kwargs = src.meta.copy()
    #print(kwargs) #print if you want to see settings of the original/reference image
    kwargs.update({
        #'height': window.height, #set to match updated window extents after cropping
        #'width': window.width, #set to match updated window extents after cropping
        'height':height
        'width':width
        'dtype':dtype, #set dtype to match inference image as defined above
        'count':1, #set count to match inference image
        'transform': rasterio.windows.transform(window, src.transform)})#updated the geotransform with new data window extents
    print(kwargs) #print updated keyword arguments
    
    with rasterio.open(output_raster, 'w', **kwargs) as dst: #defining output image and referencing updated kwargs
        dst.write(raster_with_data.read(window=window))#write data from inference image with settings defined in kwargs